In [1]:
from functools import partial, reduce
from os import rename, makedirs
import os
import urllib.request
import pandas as pd
import os.path
import shutil
import json
import csv
import re
import datetime
from typing import Literal

In [ ]:
# carpeta = f"../output/INFDES1/definitivos"

# csv_files = [x for x in os.listdir(carpeta) if ".csv" in x]

# for f in csv_files: 
#     new = f"{carpeta}/{f.replace('_old','')}"
#     old = f"{carpeta}/{f}"
#     os.rename(old, new)


In [2]:
def gsheet_download_csv(id: str, target: str, url_template="https://docs.google.com/spreadsheets/d/{id}/export?format=csv"):
    url = url_template.format(id=id)
    return urllib.request.urlretrieve(url, target)

gsheet_download_csv('1UyfAnRCes0OAOUR0aARfkfegTsUl4Puw', 'nomenclador.csv')
paises = pd.read_csv('./nomenclador.csv')
pais = {k:v for k,v in paises[['codigo_fundar', 'desc_fundar']].iloc}
codes_set = set(pais.keys())

lista_cambios = []
diccionario_cambios = {}

In [3]:
def match_relocate(src, target, pattern, file_list=None):
    files_to_move = file_list or os.listdir(src)

    pattern = re.compile(pattern)

    if not os.path.exists(target):
        makedirs(target)

    for filename in files_to_move:
        if pattern.match(filename):
            src_path = os.path.join(src, filename)
            dest_path = os.path.join(target, filename)

            shutil.move(src_path, dest_path)

    print(f"Files matching the pattern '{pattern}' moved to '{target}'.")

def compose2(f, g):
    def composed(*a, **kw):
        return f(g(*a, **kw))

    return composed

def compose(*fs):
    return reduce(compose2, fs)

def _chain(*fs):
    return compose(*reversed(fs))

def chain(*fs):
    lista_cambios.clear()
    return _chain(*fs)

def exportar_definitivo(archivo: str, folder:str, df: pd.DataFrame, nuevo_nombre=None):
    if not nuevo_nombre:
        nuevo_nombre = f"{folder}/definitivos/{archivo}_old.csv"

    original_file = f'{folder}/definitivos/{archivo}.csv'

    rename(original_file, nuevo_nombre)

    df.to_csv(original_file, 
              encoding='utf-8', 
              sep=',', 
              quoting=csv.QUOTE_ALL, 
              quotechar='"', 
              lineterminator='\n', 
              decimal='.', 
              index=False)
    
    return original_file, nuevo_nombre

def drop_col(df: pd.DataFrame, col, axis=1, lista_cambios = lista_cambios):
    lista_cambios.append(f"Se borró la columna {col}")
    return df.drop(col, axis=axis)

def drop_colx(col, axis=1):
    return lambda df: drop_col(df, col, axis=axis)

def normalize_countries(df: pd.DataFrame, code_col:str, new_col:str, desc_col:str, map: dict, lista_cambios = lista_cambios):
    codes_set = set(map.keys())
    today = datetime.datetime.now().strftime("%Y-%m-%d")
    diff_set = set(df[code_col].dropna()) - codes_set
    if len(diff_set)>0: 
        diff_str = ", ".join([str(x) for x in list(diff_set)])
        string = f"ERROR: Los códigos {diff_str} no se encontraron en el nomenclador vigente al {today} en la columna {code_col}"
        lista_cambios.append(string)
    else:
        if (new_col and desc_col):
            df[new_col] = df[code_col].apply(map.__getitem__)
            df = df.drop(desc_col, axis=1)
            string = f"Usando los valores de la columna {code_col}, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al {today}, creando la columna {new_col} y borrando la columna {desc_col}"
        elif desc_col:
            df[desc_col] = df[code_col].apply(map.__getitem__)
            string = f"Usando los valores de la columna {code_col}, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al {today}, cambiando los valores de la columna {desc_col}"
        else: 
            string = f"Usando los valores de la columna {code_col}, se verificó que todos los códigos se encuentran en el nomenclador vigente al {today}"
        lista_cambios.append(string)
    return df

def normalize_countriesx(code_col, new_col, desc_col, map):
    return lambda df: normalize_countries(df, code_col=code_col, new_col=new_col, desc_col=desc_col, map=map)

def wide_to_long(df: pd.DataFrame, primary_keys, value_name='valor', var_name='indicador', lista_cambios=lista_cambios):
    cols = ", ".join(df.columns)
    new_cols = ", ".join(primary_keys + [value_name, var_name])
    string = f"Long estricto: [{cols}] -> [{new_cols}]"
    lista_cambios.append(string)
    return df.melt(id_vars=primary_keys, value_name=value_name, var_name=var_name)

def wide_to_longx(primary_keys, value_name='valor', var_name='indicador'):
    return lambda df: wide_to_long(df, primary_keys, value_name=value_name, var_name=var_name)

def rename_cols(df: pd.DataFrame, map, lista_cambios=lista_cambios):
    map_string = ", ".join(f"{old_col}:{new_col}" for old_col, new_col in map.items())
    string = f"Se renombraron las columnas: {map_string}"
    lista_cambios.append(string)
    df = df.rename(columns=map)
    return df

def rename_colsx(map):
    return lambda df: rename_cols(df, map)

def replace_value(df:pd.DataFrame, col:str, curr_value:str, new_value:str, lista_cambios=lista_cambios):
    string = f"Se imputó el valor {new_value} en la columna {col} cuando {col}=={curr_value}"
    lista_cambios.append(string)
    df = df.replace({col: curr_value}, new_value)
    return df

def replace_valuex(col, curr_value, new_value):
    return lambda df: replace_value(df=df, col=col, curr_value=curr_value, new_value=new_value)

def sort_vals_asc(df:pd.DataFrame, prim_keys:list, lista_cambios=lista_cambios):
    lista_cambios.append(f"Ordené las filas por las columnas: {', '.join(prim_keys)}")
    return df.sort_values(by=prim_keys).reset_index(drop=True)

def sort_vals_ascx(prim_keys):
    return lambda df: sort_vals_asc(df=df, prim_keys=prim_keys)

def cast_col(df:pd.DataFrame, col:str, cast_to:Literal['int','float','object'], lista_cambios=lista_cambios):
    old_type = str(df[col].dtype)
    df[col] = df[col].astype(cast_to)
    lista_cambios.append(f"El tipo de dato de la columna {col} fue cambiado {old_type} ->> {cast_to}")
    return df

def cast_colx(col, cast_to):
    return lambda df: cast_col(df=df, col=col, cast_to=cast_to)

INFDES_g1.csv

In [4]:
SUBTOP = 'INFDES'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 1

archivo = f'{SUBTOP}_g{grafico_n}'
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

iso3 anio tipo_informalidad


,iso3,pais,anio,tipo_informalidad,valor
0,ARG,Argentina,2022,Informalidad (definición legal),36.600000
1,ARG,Argentina,2022,Informalidad (definición productiva),40.100000
2,BOL,Bolivia,2021,Informalidad (definición productiva),74.743654
3,BOL,Bolivia,2021,Informalidad (definición legal),53.709129
4,BRA,Brasil,2022,Informalidad (definición productiva),41.343411


In [8]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)


pipeline = chain(
    to_long(primary_keys + ['pais'], value_name="_valor"),
    sort_vals_ascx(primary_keys),
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    renombrar({'_valor': 'valor'}),
    nomenclar_paises(code_col='iso3', desc_col="pais", new_col=None), 
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)


diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

,iso3,anio,tipo_informalidad,pais,indicador,valor
0,ARG,2022,Informalidad (definición legal),Argentina,valor,36.600000
1,ARG,2022,Informalidad (definición productiva),Argentina,valor,40.100000
2,BOL,2021,Informalidad (definición legal),Bolivia,valor,53.709129
3,BOL,2021,Informalidad (definición productiva),Bolivia,valor,74.743654
4,BRA,2022,Informalidad (definición legal),Brasil,valor,23.577382


In [9]:
diccionario_cambios

{'INFDES_g1': ['Long estricto: [iso3, pais, anio, tipo_informalidad, valor] -> [iso3, anio, tipo_informalidad, pais, _valor, indicador]',
  'Ordené las filas por las columnas: iso3, anio, tipo_informalidad',
  'Se renombraron las columnas: _valor:valor',
  'Usando los valores de la columna iso3, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al 2024-04-10, cambiando los valores de la columna pais']}

In [10]:
exportar(_df)

('../output/INFDES1/definitivos/INFDES_g1.csv',
 '../output/INFDES1/definitivos/INFDES_g1_old.csv')

INFDES_g2.csv

In [11]:
SUBTOP = 'INFDES'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 2

archivo = f'{SUBTOP}_g{grafico_n}'
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)

df.head()


iso3 anio


,iso3,anio,pais,pib_per_capita,tasa_formalidad_productiva
0,ARG,2021,Argentina,21527.1950,0.613330
1,BOL,2021,Bolivia,8052.1772,0.252563
2,BRA,2021,Brasil,14735.5820,0.579307
3,CHL,2017,Chile,24546.9120,0.680413
4,COL,2021,Colombia,14661.2130,0.412055


In [12]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)



pipeline = chain(
    to_long(primary_keys + ['pais']),
    sort_vals_ascx(primary_keys),
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    # renombrar({'_valor': 'valor'}),
    nomenclar_paises(code_col='iso3', desc_col="pais", new_col=None), 
    #renombrar({'iso3': 'pais'}),
   
)
_df = pipeline(df)

diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

,iso3,anio,pais,indicador,valor
0,ARG,2021,Argentina,pib_per_capita,21527.195000
1,ARG,2021,Argentina,tasa_formalidad_productiva,0.613330
2,BOL,2021,Bolivia,pib_per_capita,8052.177200
3,BOL,2021,Bolivia,tasa_formalidad_productiva,0.252563
4,BRA,2021,Brasil,pib_per_capita,14735.582000


In [13]:
diccionario_cambios

{'INFDES_g1': ['Long estricto: [iso3, pais, anio, tipo_informalidad, valor] -> [iso3, anio, tipo_informalidad, pais, _valor, indicador]',
  'Ordené las filas por las columnas: iso3, anio, tipo_informalidad',
  'Se renombraron las columnas: _valor:valor',
  'Usando los valores de la columna iso3, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al 2024-04-10, cambiando los valores de la columna pais'],
 'INFDES_g2': ['Long estricto: [iso3, anio, pais, pib_per_capita, tasa_formalidad_productiva] -> [iso3, anio, pais, valor, indicador]',
  'Ordené las filas por las columnas: iso3, anio',
  'Usando los valores de la columna iso3, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al 2024-04-10, cambiando los valores de la columna pais']}

In [14]:
exportar(_df)

('../output/INFDES1/definitivos/INFDES_g2.csv',
 '../output/INFDES1/definitivos/INFDES_g2_old.csv')

INFDES_g3.csv

In [15]:
SUBTOP = 'INFDES'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 3

archivo = f'{SUBTOP}_g{grafico_n}'
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)

df.head()

iso3 anio cat_ocup_detalle


,iso3,pais,anio,cat_ocup_detalle,formal_def_productiva,valor,cat_ocup_cod
0,ARG,Argentina,2022,Asalariados en pequeñas y grandes empresas,Formal,32.4,2.0
1,ARG,Argentina,2022,Asalariados públicos,Formal,19.0,3.0
2,ARG,Argentina,2022,Cuentapropistas profesionales,Formal,4.8,4.0
3,ARG,Argentina,2022,Empleadores,Formal,3.7,1.0
4,ARG,Argentina,2022,Asalariados en microempresas,Informal,20.2,5.0


In [17]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)



pipeline = chain(
    to_long(primary_keys + ['pais','formal_def_productiva','cat_ocup_cod'], value_name="_valor"),
    sort_vals_ascx(primary_keys + ['formal_def_productiva'] ),
    nomenclar_paises(code_col='iso3', new_col=None, desc_col="pais"),
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    renombrar({'_valor': 'valor'}),
    # nomenclar_paises(code_col='cod_area', new_col="desc_area"), 
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)

diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

,iso3,anio,cat_ocup_detalle,pais,formal_def_productiva,cat_ocup_cod,indicador,valor
0,ARG,2022,Asalariados en microempresas,Argentina,Informal,5.0,valor,20.20
1,ARG,2022,Asalariados en pequeñas y grandes empresas,Argentina,Formal,2.0,valor,32.40
2,ARG,2022,Asalariados públicos,Argentina,Formal,3.0,valor,19.00
3,ARG,2022,Cuentapropistas profesionales,Argentina,Formal,4.0,valor,4.80
4,ARG,2022,Cuentapropistas sin calificación,Argentina,Informal,6.0,valor,19.35


In [18]:
diccionario_cambios

{'INFDES_g1': ['Long estricto: [iso3, pais, anio, tipo_informalidad, valor] -> [iso3, anio, tipo_informalidad, pais, _valor, indicador]',
  'Ordené las filas por las columnas: iso3, anio, tipo_informalidad',
  'Se renombraron las columnas: _valor:valor',
  'Usando los valores de la columna iso3, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al 2024-04-10, cambiando los valores de la columna pais'],
 'INFDES_g2': ['Long estricto: [iso3, anio, pais, pib_per_capita, tasa_formalidad_productiva] -> [iso3, anio, pais, valor, indicador]',
  'Ordené las filas por las columnas: iso3, anio',
  'Usando los valores de la columna iso3, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al 2024-04-10, cambiando los valores de la columna pais'],
 'INFDES_g3': ['Long estricto: [iso3, pais, anio, cat_ocup_detalle, formal_def_productiva, valor, cat_ocup_cod] -> [iso3, anio, cat_ocup_detalle, pais, formal_def_productiva, cat_ocup_cod, _valor, ind

In [19]:
exportar(_df)

('../output/INFDES1/definitivos/INFDES_g3.csv',
 '../output/INFDES1/definitivos/INFDES_g3_old.csv')

INFDES_g4.csv

In [20]:
SUBTOP = 'INFDES'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 4

archivo = f'{SUBTOP}_g{grafico_n}'
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

anio tipo_informalidad


,anio,tipo_informalidad,valor
0,2021,Definición legal,32.707735
1,2020,Definición legal,30.802683
2,2019,Definición legal,34.955574
3,2018,Definición legal,34.484882
4,2017,Definición legal,33.841891


In [21]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)



pipeline = chain(
    # nomenclar_paises(code_col='iso3c', new_col=None, desc_col='pais'),
    # drop(col="countryname"),
    to_long(primary_keys, value_name="_valor"),
    sort(primary_keys),
    renombrar({'_valor': 'valor'}),
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    # renombrar({'time': 'anio'}),
    # nomenclar_paises(code_col='cod_area', new_col="desc_area"), 
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)

diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

,anio,tipo_informalidad,indicador,valor
0,1986,Definición legal,valor,26.875632
1,1987,Definición legal,valor,29.162604
2,1988,Definición legal,valor,33.513792
3,1988,Definición productiva,valor,48.101269
4,1991,Definición legal,valor,36.317777


In [22]:
diccionario_cambios

{'INFDES_g1': ['Long estricto: [iso3, pais, anio, tipo_informalidad, valor] -> [iso3, anio, tipo_informalidad, pais, _valor, indicador]',
  'Ordené las filas por las columnas: iso3, anio, tipo_informalidad',
  'Se renombraron las columnas: _valor:valor',
  'Usando los valores de la columna iso3, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al 2024-04-10, cambiando los valores de la columna pais'],
 'INFDES_g2': ['Long estricto: [iso3, anio, pais, pib_per_capita, tasa_formalidad_productiva] -> [iso3, anio, pais, valor, indicador]',
  'Ordené las filas por las columnas: iso3, anio',
  'Usando los valores de la columna iso3, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al 2024-04-10, cambiando los valores de la columna pais'],
 'INFDES_g3': ['Long estricto: [iso3, pais, anio, cat_ocup_detalle, formal_def_productiva, valor, cat_ocup_cod] -> [iso3, anio, cat_ocup_detalle, pais, formal_def_productiva, cat_ocup_cod, _valor, ind

In [23]:
exportar(_df)

('../output/INFDES1/definitivos/INFDES_g4.csv',
 '../output/INFDES1/definitivos/INFDES_g4_old.csv')

INFDES_g5.csv

In [69]:
SUBTOP = 'INFDES'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 5

archivo = f'{SUBTOP}_g{grafico_n}'
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

iso3 anio serie


,iso3,pais,anio,serie,valor
0,ARG,Argentina,2021,Serie empalmada,0.327077
1,ARG,Argentina,2020,Serie empalmada,0.308027
2,ARG,Argentina,2019,Serie empalmada,0.349556
3,ARG,Argentina,2018,Serie empalmada,0.344849
4,ARG,Argentina,2017,Serie empalmada,0.338419


In [70]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)



pipeline = chain(
    to_long(primary_keys + ['pais'], value_name="_valor"),
    renombrar({'_valor':'valor'}),
    sort(primary_keys),
    nomenclar_paises(code_col='iso3', new_col=None, desc_col=None)
      
)

_df = pipeline(df)

diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

,iso3,anio,serie,pais,indicador,valor
0,ARG,1986,Serie empalmada,Argentina,valor,0.268756
1,ARG,1987,Serie empalmada,Argentina,valor,0.291626
2,ARG,1988,Serie empalmada,Argentina,valor,0.335138
3,ARG,1991,Serie empalmada,Argentina,valor,0.363178
4,ARG,1992,Serie empalmada,Argentina,valor,0.350231


In [71]:
diccionario_cambios

{'INFDES_g1': ['Long estricto: [iso3, pais, anio, tipo_informalidad, valor] -> [iso3, anio, tipo_informalidad, pais, _valor, indicador]',
  'Ordené las filas por las columnas: iso3, anio, tipo_informalidad',
  'Se renombraron las columnas: _valor:valor',
  'Usando los valores de la columna iso3, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al 2024-04-10, cambiando los valores de la columna pais'],
 'INFDES_g2': ['Long estricto: [iso3, anio, pais, pib_per_capita, tasa_formalidad_productiva] -> [iso3, anio, pais, valor, indicador]',
  'Ordené las filas por las columnas: iso3, anio',
  'Usando los valores de la columna iso3, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al 2024-04-10, cambiando los valores de la columna pais'],
 'INFDES_g3': ['Long estricto: [iso3, pais, anio, cat_ocup_detalle, formal_def_productiva, valor, cat_ocup_cod] -> [iso3, anio, cat_ocup_detalle, pais, formal_def_productiva, cat_ocup_cod, _valor, ind

In [72]:
exportar(_df)

('../output/INFDES1/definitivos/INFDES_g5.csv',
 '../output/INFDES1/definitivos/INFDES_g5_old.csv')

INFDES_g6.csv

In [28]:
SUBTOP = 'INFDES'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 6

archivo = f'{SUBTOP}_g{grafico_n}'
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

anio prov_cod tipo_informalidad


,anio,prov_cod,prov_desc,tipo_informalidad,valor
0,2016,2,CABA,Informalidad (definición productiva),0.262605
1,2017,2,CABA,Informalidad (definición productiva),0.259833
2,2018,2,CABA,Informalidad (definición productiva),0.302962
3,2019,2,CABA,Informalidad (definición productiva),0.269595
4,2020,2,CABA,Informalidad (definición productiva),0.258505


In [29]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)



pipeline = chain(
    to_long(primary_keys + ['prov_desc'], value_name="_valor"),
    sort(primary_keys),
    renombrar({"_valor":"valor"})
)

_df = pipeline(df)

diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

,anio,prov_cod,tipo_informalidad,prov_desc,indicador,valor
0,2016,-9,Informalidad (definición legal),Total país,valor,0.356503
1,2016,-9,Informalidad (definición productiva),Total país,valor,0.436331
2,2016,2,Informalidad (definición legal),CABA,valor,0.208920
3,2016,2,Informalidad (definición productiva),CABA,valor,0.262605
4,2016,6,Informalidad (definición legal),Buenos Aires,valor,0.365511


In [30]:
diccionario_cambios

{'INFDES_g1': ['Long estricto: [iso3, pais, anio, tipo_informalidad, valor] -> [iso3, anio, tipo_informalidad, pais, _valor, indicador]',
  'Ordené las filas por las columnas: iso3, anio, tipo_informalidad',
  'Se renombraron las columnas: _valor:valor',
  'Usando los valores de la columna iso3, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al 2024-04-10, cambiando los valores de la columna pais'],
 'INFDES_g2': ['Long estricto: [iso3, anio, pais, pib_per_capita, tasa_formalidad_productiva] -> [iso3, anio, pais, valor, indicador]',
  'Ordené las filas por las columnas: iso3, anio',
  'Usando los valores de la columna iso3, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al 2024-04-10, cambiando los valores de la columna pais'],
 'INFDES_g3': ['Long estricto: [iso3, pais, anio, cat_ocup_detalle, formal_def_productiva, valor, cat_ocup_cod] -> [iso3, anio, cat_ocup_detalle, pais, formal_def_productiva, cat_ocup_cod, _valor, ind

In [31]:
exportar(_df)

('../output/INFDES1/definitivos/INFDES_g6.csv',
 '../output/INFDES1/definitivos/INFDES_g6_old.csv')

INFDES_g7.csv

In [32]:
SUBTOP = 'INFDES'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 7

archivo = f'{SUBTOP}_g{grafico_n}'
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

anio edad apertura_sexo


,anio,edad,apertura_sexo,tasa_informalidad_legal
0,2016,18,femenino,0.945639
1,2016,19,femenino,0.914439
2,2016,20,femenino,0.813144
3,2016,21,femenino,0.775637
4,2016,22,femenino,0.711833


In [33]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)



pipeline = chain(
    to_long(primary_keys),
    sort(primary_keys)
   
)

_df = pipeline(df)

diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

,anio,edad,apertura_sexo,indicador,valor
0,2016,18,femenino,tasa_informalidad_legal,0.945639
1,2016,18,masculino,tasa_informalidad_legal,0.908279
2,2016,18,total,tasa_informalidad_legal,0.919120
3,2016,19,femenino,tasa_informalidad_legal,0.914439
4,2016,19,masculino,tasa_informalidad_legal,0.877820


In [34]:
diccionario_cambios

{'INFDES_g1': ['Long estricto: [iso3, pais, anio, tipo_informalidad, valor] -> [iso3, anio, tipo_informalidad, pais, _valor, indicador]',
  'Ordené las filas por las columnas: iso3, anio, tipo_informalidad',
  'Se renombraron las columnas: _valor:valor',
  'Usando los valores de la columna iso3, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al 2024-04-10, cambiando los valores de la columna pais'],
 'INFDES_g2': ['Long estricto: [iso3, anio, pais, pib_per_capita, tasa_formalidad_productiva] -> [iso3, anio, pais, valor, indicador]',
  'Ordené las filas por las columnas: iso3, anio',
  'Usando los valores de la columna iso3, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al 2024-04-10, cambiando los valores de la columna pais'],
 'INFDES_g3': ['Long estricto: [iso3, pais, anio, cat_ocup_detalle, formal_def_productiva, valor, cat_ocup_cod] -> [iso3, anio, cat_ocup_detalle, pais, formal_def_productiva, cat_ocup_cod, _valor, ind

In [35]:
exportar(_df)

('../output/INFDES1/definitivos/INFDES_g7.csv',
 '../output/INFDES1/definitivos/INFDES_g7_old.csv')

INFDES_g8.csv

In [36]:
SUBTOP = 'INFDES'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 8

archivo = f'{SUBTOP}_g{grafico_n}'
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

anio apertura_sexo


,anio,apertura_sexo,valor
0,1986,Mujer,0.299352
1,1987,Mujer,0.326684
2,1988,Mujer,0.359710
3,1991,Mujer,0.344463
4,1992,Mujer,0.342886


In [37]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)



pipeline = chain(
    to_long(primary_keys, value_name="_valor"),
    sort(primary_keys),
    # renombrar({'iso3': 'cod_fundar','time':'anio'}),
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    renombrar({'_valor': 'valor'}),
    # nomenclar_paises(code_col='cod_area', new_col="desc_area"), 
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)

diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

,anio,apertura_sexo,indicador,valor
0,1986,Brecha,valor,-0.161488
1,1986,Mujer,valor,0.299352
2,1986,Varón,valor,0.137864
3,1987,Brecha,valor,-0.175542
4,1987,Mujer,valor,0.326684


In [38]:
diccionario_cambios

{'INFDES_g1': ['Long estricto: [iso3, pais, anio, tipo_informalidad, valor] -> [iso3, anio, tipo_informalidad, pais, _valor, indicador]',
  'Ordené las filas por las columnas: iso3, anio, tipo_informalidad',
  'Se renombraron las columnas: _valor:valor',
  'Usando los valores de la columna iso3, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al 2024-04-10, cambiando los valores de la columna pais'],
 'INFDES_g2': ['Long estricto: [iso3, anio, pais, pib_per_capita, tasa_formalidad_productiva] -> [iso3, anio, pais, valor, indicador]',
  'Ordené las filas por las columnas: iso3, anio',
  'Usando los valores de la columna iso3, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al 2024-04-10, cambiando los valores de la columna pais'],
 'INFDES_g3': ['Long estricto: [iso3, pais, anio, cat_ocup_detalle, formal_def_productiva, valor, cat_ocup_cod] -> [iso3, anio, cat_ocup_detalle, pais, formal_def_productiva, cat_ocup_cod, _valor, ind

In [39]:
exportar(_df)

('../output/INFDES1/definitivos/INFDES_g8.csv',
 '../output/INFDES1/definitivos/INFDES_g8_old.csv')

INFDES_g9.csv

In [49]:
SUBTOP = 'INFDES'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 9

archivo = f'{SUBTOP}_g{grafico_n}'
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

iso3 anio_circa circa


,iso3,pais,anio_circa,brecha,circa
0,ARG,Argentina,2000,13.99,2000
1,BRA,Brasil,1999,3.00,2000
2,BOL,Bolivia,2000,3.70,2000
3,CHL,Chile,2000,6.00,2000
4,CRI,Costa Rica,2000,8.50,2000


In [50]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)



pipeline = chain(
    to_long(primary_keys + ['pais']),
    sort(primary_keys),
    # renombrar({'iso3': 'cod_fundar','time':'anio'}),
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    # renombrar({'time': 'anio'}),
    nomenclar_paises(code_col='iso3', desc_col="pais", new_col=None), 
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)

diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

,iso3,anio_circa,circa,pais,indicador,valor
0,ARG,2000,2000,Argentina,brecha,13.99
1,ARG,2021,2021,Argentina,brecha,5.60
2,BOL,2000,2000,Bolivia,brecha,3.70
3,BOL,2021,2021,Bolivia,brecha,4.30
4,BRA,1999,2000,Brasil,brecha,3.00


In [51]:
diccionario_cambios

{'INFDES_g1': ['Long estricto: [iso3, pais, anio, tipo_informalidad, valor] -> [iso3, anio, tipo_informalidad, pais, _valor, indicador]',
  'Ordené las filas por las columnas: iso3, anio, tipo_informalidad',
  'Se renombraron las columnas: _valor:valor',
  'Usando los valores de la columna iso3, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al 2024-04-10, cambiando los valores de la columna pais'],
 'INFDES_g2': ['Long estricto: [iso3, anio, pais, pib_per_capita, tasa_formalidad_productiva] -> [iso3, anio, pais, valor, indicador]',
  'Ordené las filas por las columnas: iso3, anio',
  'Usando los valores de la columna iso3, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al 2024-04-10, cambiando los valores de la columna pais'],
 'INFDES_g3': ['Long estricto: [iso3, pais, anio, cat_ocup_detalle, formal_def_productiva, valor, cat_ocup_cod] -> [iso3, anio, cat_ocup_detalle, pais, formal_def_productiva, cat_ocup_cod, _valor, ind

In [52]:
exportar(_df)

('../output/INFDES1/definitivos/INFDES_g9.csv',
 '../output/INFDES1/definitivos/INFDES_g9_old.csv')

INFDES_g10.csv

In [59]:
SUBTOP = 'INFDES'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 10

archivo = f'{SUBTOP}_g{grafico_n}'
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

iso3 anio apertura_sexo


,iso3,anio,apertura_sexo,valor,pais
0,ARG,2021,femenino,12.345976,Argentina
1,ARG,2020,femenino,11.963503,Argentina
2,ARG,2019,femenino,11.953631,Argentina
3,ARG,2018,femenino,11.851245,Argentina
4,ARG,2017,femenino,11.715534,Argentina


In [60]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)



pipeline = chain(
    to_long(['iso3','pais','anio','apertura_sexo'], value_name="_valor"),
    sort(primary_keys),
    # renombrar({'iso3': 'cod_fundar','time':'anio'}),
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    renombrar({'_valor': 'valor'}),
    nomenclar_paises(code_col='iso3', new_col=None, desc_col=None), 
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)

diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

,iso3,pais,anio,apertura_sexo,indicador,valor
0,ARG,Argentina,1980,femenino,valor,8.137108
1,ARG,Argentina,1980,masculino,valor,8.504015
2,ARG,Argentina,1986,femenino,valor,8.884459
3,ARG,Argentina,1986,masculino,valor,9.305058
4,ARG,Argentina,1988,femenino,valor,9.044661


In [61]:
diccionario_cambios

{'INFDES_g1': ['Long estricto: [iso3, pais, anio, tipo_informalidad, valor] -> [iso3, anio, tipo_informalidad, pais, _valor, indicador]',
  'Ordené las filas por las columnas: iso3, anio, tipo_informalidad',
  'Se renombraron las columnas: _valor:valor',
  'Usando los valores de la columna iso3, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al 2024-04-10, cambiando los valores de la columna pais'],
 'INFDES_g2': ['Long estricto: [iso3, anio, pais, pib_per_capita, tasa_formalidad_productiva] -> [iso3, anio, pais, valor, indicador]',
  'Ordené las filas por las columnas: iso3, anio',
  'Usando los valores de la columna iso3, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al 2024-04-10, cambiando los valores de la columna pais'],
 'INFDES_g3': ['Long estricto: [iso3, pais, anio, cat_ocup_detalle, formal_def_productiva, valor, cat_ocup_cod] -> [iso3, anio, cat_ocup_detalle, pais, formal_def_productiva, cat_ocup_cod, _valor, ind

In [62]:
exportar(_df)

('../output/INFDES1/definitivos/INFDES_g10.csv',
 '../output/INFDES1/definitivos/INFDES_g10_old.csv')

INFDES_g11.csv

In [73]:
SUBTOP = 'INFDES'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 11

archivo = f'{SUBTOP}_g{grafico_n}'
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

iso3 anio


,iso3,pais_desc,anio,tasa_desempleo
0,ARG,Argentina,1991,0.0544
1,ARG,Argentina,1992,0.0636
2,ARG,Argentina,1993,0.1010
3,ARG,Argentina,1994,0.1176
4,ARG,Argentina,1995,0.1880


In [76]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)



pipeline = chain(
    to_long(primary_keys + ['pais_desc']),
    sort(primary_keys),
    # renombrar({'iso3': 'cod_fundar','time':'anio'}),
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    # renombrar({'time': 'anio'}),
    nomenclar_paises(code_col='iso3', new_col=None, desc_col="pais_desc"), 
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)

diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

,iso3,anio,pais_desc,indicador,valor
0,ARG,1991,Argentina,tasa_desempleo,0.0544
1,ARG,1992,Argentina,tasa_desempleo,0.0636
2,ARG,1993,Argentina,tasa_desempleo,0.1010
3,ARG,1994,Argentina,tasa_desempleo,0.1176
4,ARG,1995,Argentina,tasa_desempleo,0.1880


In [77]:
diccionario_cambios

{'INFDES_g1': ['Long estricto: [iso3, pais, anio, tipo_informalidad, valor] -> [iso3, anio, tipo_informalidad, pais, _valor, indicador]',
  'Ordené las filas por las columnas: iso3, anio, tipo_informalidad',
  'Se renombraron las columnas: _valor:valor',
  'Usando los valores de la columna iso3, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al 2024-04-10, cambiando los valores de la columna pais'],
 'INFDES_g2': ['Long estricto: [iso3, anio, pais, pib_per_capita, tasa_formalidad_productiva] -> [iso3, anio, pais, valor, indicador]',
  'Ordené las filas por las columnas: iso3, anio',
  'Usando los valores de la columna iso3, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al 2024-04-10, cambiando los valores de la columna pais'],
 'INFDES_g3': ['Long estricto: [iso3, pais, anio, cat_ocup_detalle, formal_def_productiva, valor, cat_ocup_cod] -> [iso3, anio, cat_ocup_detalle, pais, formal_def_productiva, cat_ocup_cod, _valor, ind

In [78]:
exportar(_df)

('../output/INFDES1/definitivos/INFDES_g11.csv',
 '../output/INFDES1/definitivos/INFDES_g11_old.csv')

INFDES_g12.csv

In [79]:
SUBTOP = 'INFDES'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 12

archivo = f'{SUBTOP}_g{grafico_n}'
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

iso3 anio sexo


,iso3,pais_desc,continente_fundar,anio,sexo,tasa_desempleo
0,ABW,Aruba,"América del Norte, Central y el Caribe",1991,Total,NaN
1,ABW,Aruba,"América del Norte, Central y el Caribe",1992,Total,NaN
2,ABW,Aruba,"América del Norte, Central y el Caribe",1993,Total,NaN
3,ABW,Aruba,"América del Norte, Central y el Caribe",1994,Total,NaN
4,ABW,Aruba,"América del Norte, Central y el Caribe",1995,Total,NaN


In [80]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)



pipeline = chain(
    to_long(['anio', 'iso3', 'pais_desc','continente_fundar', 'sexo']),
    sort(['anio', 'iso3', 'pais_desc','continente_fundar', 'sexo']),
    # renombrar({'iso3': 'cod_fundar','time':'anio'}),
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    # renombrar({'time': 'anio'}),
    nomenclar_paises(code_col='iso3', new_col=None, desc_col="pais_desc"), 
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)

diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

,anio,iso3,pais_desc,continente_fundar,sexo,indicador,valor
0,1991,ABW,Aruba,"América del Norte, Central y el Caribe",Mujeres,tasa_desempleo,NaN
1,1991,ABW,Aruba,"América del Norte, Central y el Caribe",Total,tasa_desempleo,NaN
2,1991,ABW,Aruba,"América del Norte, Central y el Caribe",Varones,tasa_desempleo,NaN
3,1991,AFG,Afganistán,Asia,Mujeres,tasa_desempleo,0.10433
4,1991,AFG,Afganistán,Asia,Total,tasa_desempleo,0.07942


In [81]:
diccionario_cambios

{'INFDES_g1': ['Long estricto: [iso3, pais, anio, tipo_informalidad, valor] -> [iso3, anio, tipo_informalidad, pais, _valor, indicador]',
  'Ordené las filas por las columnas: iso3, anio, tipo_informalidad',
  'Se renombraron las columnas: _valor:valor',
  'Usando los valores de la columna iso3, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al 2024-04-10, cambiando los valores de la columna pais'],
 'INFDES_g2': ['Long estricto: [iso3, anio, pais, pib_per_capita, tasa_formalidad_productiva] -> [iso3, anio, pais, valor, indicador]',
  'Ordené las filas por las columnas: iso3, anio',
  'Usando los valores de la columna iso3, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al 2024-04-10, cambiando los valores de la columna pais'],
 'INFDES_g3': ['Long estricto: [iso3, pais, anio, cat_ocup_detalle, formal_def_productiva, valor, cat_ocup_cod] -> [iso3, anio, cat_ocup_detalle, pais, formal_def_productiva, cat_ocup_cod, _valor, ind

In [83]:
exportar(_df)

('../output/INFDES1/definitivos/INFDES_g12.csv',
 '../output/INFDES1/definitivos/INFDES_g12_old.csv')

INFDES_g13.csv

In [84]:
SUBTOP = 'INFDES'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 13

archivo = f'{SUBTOP}_g{grafico_n}'
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

iso3 anio estado


,iso3,pais_desc,anio,estado,satisfaccion_vida
0,AND,Andorra,2018,Ocupado,7.592547
1,AND,Andorra,2018,Desocupado,7.466667
2,ALB,Albania,2018,Ocupado,7.212840
3,ALB,Albania,2018,Desocupado,6.642002
4,ARM,Armenia,2018,Ocupado,6.346251


In [85]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)



pipeline = chain(
    to_long(['anio', 'iso3', 'pais_desc', 'estado']),
    sort(['anio', 'iso3', 'pais_desc', 'estado']),
    # renombrar({'iso3': 'cod_fundar','time':'anio'}),
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    # renombrar({'time': 'anio'}),
    nomenclar_paises(code_col='iso3', new_col=None, desc_col="pais_desc"), 
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)

diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

,anio,iso3,pais_desc,estado,indicador,valor
0,2017,ARG,Argentina,Desocupado,satisfaccion_vida,6.691225
1,2017,ARG,Argentina,Ocupado,satisfaccion_vida,7.652228
2,2017,BGR,Bulgaria,Desocupado,satisfaccion_vida,6.140818
3,2017,BGR,Bulgaria,Ocupado,satisfaccion_vida,6.856316
4,2017,BOL,Bolivia,Desocupado,satisfaccion_vida,7.480769


In [86]:
diccionario_cambios

{'INFDES_g1': ['Long estricto: [iso3, pais, anio, tipo_informalidad, valor] -> [iso3, anio, tipo_informalidad, pais, _valor, indicador]',
  'Ordené las filas por las columnas: iso3, anio, tipo_informalidad',
  'Se renombraron las columnas: _valor:valor',
  'Usando los valores de la columna iso3, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al 2024-04-10, cambiando los valores de la columna pais'],
 'INFDES_g2': ['Long estricto: [iso3, anio, pais, pib_per_capita, tasa_formalidad_productiva] -> [iso3, anio, pais, valor, indicador]',
  'Ordené las filas por las columnas: iso3, anio',
  'Usando los valores de la columna iso3, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al 2024-04-10, cambiando los valores de la columna pais'],
 'INFDES_g3': ['Long estricto: [iso3, pais, anio, cat_ocup_detalle, formal_def_productiva, valor, cat_ocup_cod] -> [iso3, anio, cat_ocup_detalle, pais, formal_def_productiva, cat_ocup_cod, _valor, ind

In [87]:
exportar(_df)

('../output/INFDES1/definitivos/INFDES_g13.csv',
 '../output/INFDES1/definitivos/INFDES_g13_old.csv')

INFDES_g14.csv

In [88]:
SUBTOP = 'INFDES'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 14

archivo = f'{SUBTOP}_g{grafico_n}'
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

anio rango_edad sexo


,anio,rango_edad,rango_edad_desc,sexo,tasa_desocupacion
0,2016,1,Hasta 30,Mujeres,0.193575
1,2016,2,Entre 30 y 39,Mujeres,0.061645
2,2016,3,Entre 40 y 49,Mujeres,0.053957
3,2016,4,50 y más,Mujeres,0.028682
4,2016,1,Hasta 30,Varones,0.133230


In [89]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)



pipeline = chain(
    to_long(['anio', 'rango_edad', 'rango_edad_desc','sexo']),
    sort(primary_keys),
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    # renombrar({'time': 'anio'}),
    # nomenclar_paises(code_col='cod_area', new_col="desc_area"), 
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)

diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

,anio,rango_edad,rango_edad_desc,sexo,indicador,valor
0,2016,1,Hasta 30,Mujeres,tasa_desocupacion,0.193575
1,2016,1,Hasta 30,Varones,tasa_desocupacion,0.133230
2,2016,2,Entre 30 y 39,Mujeres,tasa_desocupacion,0.061645
3,2016,2,Entre 30 y 39,Varones,tasa_desocupacion,0.060592
4,2016,3,Entre 40 y 49,Mujeres,tasa_desocupacion,0.053957


In [90]:
diccionario_cambios

{'INFDES_g1': ['Long estricto: [iso3, pais, anio, tipo_informalidad, valor] -> [iso3, anio, tipo_informalidad, pais, _valor, indicador]',
  'Ordené las filas por las columnas: iso3, anio, tipo_informalidad',
  'Se renombraron las columnas: _valor:valor',
  'Usando los valores de la columna iso3, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al 2024-04-10, cambiando los valores de la columna pais'],
 'INFDES_g2': ['Long estricto: [iso3, anio, pais, pib_per_capita, tasa_formalidad_productiva] -> [iso3, anio, pais, valor, indicador]',
  'Ordené las filas por las columnas: iso3, anio',
  'Usando los valores de la columna iso3, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al 2024-04-10, cambiando los valores de la columna pais'],
 'INFDES_g3': ['Long estricto: [iso3, pais, anio, cat_ocup_detalle, formal_def_productiva, valor, cat_ocup_cod] -> [iso3, anio, cat_ocup_detalle, pais, formal_def_productiva, cat_ocup_cod, _valor, ind

In [91]:
exportar(_df)

('../output/INFDES1/definitivos/INFDES_g14.csv',
 '../output/INFDES1/definitivos/INFDES_g14_old.csv')

INFDES_g15.csv

In [92]:
SUBTOP = 'INFDES'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 15

archivo = f'{SUBTOP}_g{grafico_n}'
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

anio nivel_ed_cod


,anio,nivel_ed_cod,nivel_ed_desc,tasa_desocupacion
0,2003,1,Hasta secundaria incompleta,0.119194
1,2003,2,Secundaria completa,0.101719
2,2003,4,Superior completa,0.046821
3,2003,3,Superior incompleta,0.096215
4,2004,1,Hasta secundaria incompleta,0.104605


In [93]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)



pipeline = chain(
    to_long(primary_keys + ['nivel_ed_desc']),
    sort(primary_keys),
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    # renombrar({'time': 'anio'}),
    # nomenclar_paises(code_col='cod_area', new_col="desc_area"), 
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)

diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

,anio,nivel_ed_cod,nivel_ed_desc,indicador,valor
0,2003,1,Hasta secundaria incompleta,tasa_desocupacion,0.119194
1,2003,2,Secundaria completa,tasa_desocupacion,0.101719
2,2003,3,Superior incompleta,tasa_desocupacion,0.096215
3,2003,4,Superior completa,tasa_desocupacion,0.046821
4,2004,1,Hasta secundaria incompleta,tasa_desocupacion,0.104605


In [94]:
diccionario_cambios

{'INFDES_g1': ['Long estricto: [iso3, pais, anio, tipo_informalidad, valor] -> [iso3, anio, tipo_informalidad, pais, _valor, indicador]',
  'Ordené las filas por las columnas: iso3, anio, tipo_informalidad',
  'Se renombraron las columnas: _valor:valor',
  'Usando los valores de la columna iso3, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al 2024-04-10, cambiando los valores de la columna pais'],
 'INFDES_g2': ['Long estricto: [iso3, anio, pais, pib_per_capita, tasa_formalidad_productiva] -> [iso3, anio, pais, valor, indicador]',
  'Ordené las filas por las columnas: iso3, anio',
  'Usando los valores de la columna iso3, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al 2024-04-10, cambiando los valores de la columna pais'],
 'INFDES_g3': ['Long estricto: [iso3, pais, anio, cat_ocup_detalle, formal_def_productiva, valor, cat_ocup_cod] -> [iso3, anio, cat_ocup_detalle, pais, formal_def_productiva, cat_ocup_cod, _valor, ind

In [95]:
exportar(_df)

('../output/INFDES1/definitivos/INFDES_g15.csv',
 '../output/INFDES1/definitivos/INFDES_g15_old.csv')

In [96]:
SUBTOP = 'INFDES'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
match_relocate(f'{folder}/definitivos', f'{folder}/old', '.*old.*')

Files matching the pattern 're.compile('.*old.*')' moved to '../output/INFDES1/old'.


In [97]:
import json
import datetime

today = datetime.datetime.now().strftime("%Y-%m-%d")

with open(f'../output/INFDES1/definitivos/paso_a_definitivos_{today}.json', 'w') as fp:
    json.dump(diccionario_cambios, fp, indent=4, ensure_ascii=False)
